**Step 1: Import Lib**

In [2]:
import numpy as np
import matplotlib.pylab as plt
from scipy.misc import imresize, imread
import itertools
import sklearn
import cv2
import time
import shutil
import os
from scipy.misc import imresize
from skimage import transform,data
from sklearn import model_selection
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, learning_curve, GridSearchCV
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import keras
from keras import backend as K
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, model_from_json
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta, RMSprop
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D, Input
from keras.models import Sequential, Model
from keras.preprocessing import image
from keras.callbacks import TensorBoard
from keras.utils import plot_model
from keras.models import Model
%matplotlib inline
plt.style.use({'figure.figsize':(50,50)})

**Step 2: Import Image**

In [3]:
X = cv2.imread('../input/Pic/example.png')
X1 = cv2.imread('../input/Pic/example.png')
#X = X.reshape((1,) + X.shape)

In [4]:
"""
plt.imshow(image) # 显示图片(if you want)
plt.axis('on') # 显示坐标轴
plt.show()
"""

"\nplt.imshow(image) # 显示图片(if you want)\nplt.axis('on') # 显示坐标轴\nplt.show()\n"

In [6]:
X = np.load('../input/X.npy') # images
Y = np.load('../input/Y.npy') # labels associated to images (0 = non-tumor, 1 = Tumor)

In [7]:
Y_reshape = to_categorical(Y,num_classes = 2)
Y_reshape.shape

Y = Y_reshape

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

# Reduce Sample Size for DeBugging
X_train = X_train[0:30000] 
Y_train =Y_train[0:30000]
X_test = X_test[0:30000] 
Y_test = Y_test[0:30000]

# Normalize the data
X_train = X_train / 256.0
X_test = X_test / 256.0

print("Training Data Shape:", X_train.shape, X_train.shape)
print("Testing Data Shape:", X_test.shape, X_test.shape)

Training Data Shape: (4437, 50, 50, 3) (4437, 50, 50, 3)
Testing Data Shape: (1110, 50, 50, 3) (1110, 50, 50, 3)


**Step 3: Identify IDC+ or IDC-**

In [10]:
batch_size = 128
num_classes = 2
epochs = 300
img_rows, img_cols = X.shape[1],X.shape[2]
input_shape = (img_rows, img_cols, 3)


model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
print(model.summary())
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

datagen = ImageDataGenerator(featurewise_center=False,  # set input mean to 0 over the dataset 
                             samplewise_center=False,  # set each sample mean to 0
                             featurewise_std_normalization=False,  # divide inputs by std of the dataset
                             samplewise_std_normalization=False,  # divide each input by its std
                             zca_whitening=False,  # apply ZCA whitening
                             rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
                             width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
                             height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
                             horizontal_flip=True,  # randomly flip images
                             vertical_flip=True)  # randomly flip images

proghistory = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=32),
                    steps_per_epoch=len(X_train) / 32, epochs=epochs,validation_data = [X_test,Y_test],
                    callbacks=[TensorBoard(log_dir='mytensorboard')])

preds = model.predict(X)
T = preds[0][0]
F = preds[0][1]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 48, 48, 32)        896       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 46, 46, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 21, 21, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6400)              0         
__________

KeyboardInterrupt: 

In [ ]:
def plot_confusion_matrix(cm, title='Confusion Matrix', cmap=plt.cm.binary,classes=None):
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    xlocations = np.array(range(len(classes)))
    plt.xticks(xlocations, classes, rotation=90)
    plt.yticks(xlocations, classes)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
score = model.evaluate(X_test,Y_test, verbose=0)
print('\nKeras CNN #1C - accuracy:', score[1],'\n')
y_pred = model.predict(X_test)
map_characters = {0: 'Tumor(-)', 1: 'Tumor(+)'}
print('\n', sklearn.metrics.classification_report(np.where(Y_test > 0)[1], np.argmax(y_pred, axis=1), target_names=list(map_characters.values())), sep='')    
score = model.evaluate(X_test,Y_test, verbose=0)
Y_pred_classes = np.argmax(y_pred,axis = 1) 
Y_true = np.argmax(Y_test,axis = 1) 
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
plot_confusion_matrix(confusion_mtx, classes = list(map_characters.values()))  

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def plot_history(proghistory):
    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.plot(proghistory.history['loss'])
    plt.plot(proghistory.history['val_loss'])
    plt.legend(['Training', 'Validation'])

    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.plot(proghistory.history['acc'])
    plt.plot(proghistory.history['val_acc'])
    plt.legend(['Training', 'Validation'], loc='lower right')
    plt.show()

plot_history(proghistory)

batch_size = 100
num_classes = 2
epochs = 500

def vgg16_model(input_shape = (50,50,3)):
    vgg16 = VGG16(include_top = False, weights = 'imagenet', input_shape = input_shape)
    
    for layer in vgg16.layers:
        layer.trainable = False
    last = vgg16.output
   
    x = Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape)
    x = Conv2D(64, (3, 3), activation='relu')
    x = MaxPooling2D(pool_size=(2, 2))
    x = Conv2D(128, (3, 3), activation='relu')
    x = Dropout(0.25)
    x = Conv2D(256, (3, 3), activation='relu')
    x = MaxPooling2D(pool_size=(2, 2))
    x = Dropout(0.25)
    x = Flatten()(last)
    x = Dense(256, activation = 'relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation = 'relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(2, activation = 'softmax')(x)
    
    model = Model(inputs = vgg16.input, outputs = x)
    
    return model

datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images


vgg16 = vgg16_model()
vgg16.compile(loss = keras.losses.categorical_crossentropy, optimizer = keras.optimizers.Adadelta(), metrics = ['accuracy'])
vgg16.fit_generator(datagen.flow(X_train,Y_train, batch_size=20),
                        steps_per_epoch=len(X_train) / 30, epochs=epochs,validation_data = [X_test,Y_test])


model_vgg16 = vgg16_model()
model_vgg16.summary()
model_vgg16.compile(loss = 'categorical_crossentropy', optimizer = Adam(0.0001), metrics = ['accuracy'])

In [ ]:
m = 0
r = 100
for i in range(r):
    p = model.predict(np.expand_dims(X_test[i],axis=0))
    a = Y_test[i]
    print(p,a)
    if p[0][0]< p[0][1] and a[0]< a[1]:
        m = m+1
    elif p[0][0]> p[0][1] and a[0] > a[1]:
        m = m+1
    
    print(m/r)

In [ ]:
preds = model.predict(X)
T = preds[0][0]
F = preds[0][1]

**Step 4: Generate Reports**

In [ ]:
if T>F:
    im1 = X1
    Img = cv2.resize(im1, (256,256), interpolation=cv2.INTER_CUBIC)
    kernel_2 = np.ones((2, 2), np.uint8)  # 2x2的卷积核
    kernel_3 = np.ones((3, 3), np.uint8)  # 3x3的卷积核
    kernel_4 = np.ones((4, 4), np.uint8)  # 4x4的卷积核
    if Img is not None:  # 判断图片是否读入
        HSV = cv2.cvtColor(Img, cv2.COLOR_BGR2HSV)  # 把RGB图像转换为HSV格式
        Lower = np.array([90, 25, 0])  # 要识别颜色的下限
        Upper = np.array([154, 255, 225])  # 要识别的颜色的上限
        # mask是把HSV图片中在颜色范围内的区域变成白色，其他区域变成黑色
        mask = cv2.inRange(HSV, Lower, Upper)
        # 下面四行是用卷积进行滤波
        erosion = cv2.erode(mask, kernel_4, iterations=1)
        erosion = cv2.erode(erosion, kernel_4, iterations=1)
        dilation = cv2.dilate(erosion, kernel_4, iterations=1)
        dilation = cv2.dilate(dilation, kernel_4, iterations=1)
        # target是把原图中的非目标颜色区域去掉剩下的图像
        target = cv2.bitwise_and(Img, Img, mask=dilation)
        # 将滤波后的图像变成二值图像放在binary中
        ret, binary = cv2.threshold(dilation, 127, 255, cv2.THRESH_BINARY)
        # 在binary中发现轮廓，轮廓按照面积从小到大排列
        contours, hierarchy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        p = 0
        for i in contours:  # 遍历所有的轮廓
            x, y, w, h = cv2.boundingRect(i)  # 将轮廓分解为识别对象的左上角坐标和宽、高
            # 在图像上画上矩形（图片、左上角坐标、右下角坐标、颜色、线条宽度）
            cv2.rectangle(Img, (x, y), (x + w, y + h), (0, 255,), 2)
            # 给识别对象写上标号
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(Img, str(p), (x - 10, y + 10), font, 0.5, (0, 0, 255), 1)  # 加减10是调整字符位置
            p += 1
        print('There are', p, 'potential tumor areas')  # 终端输出目标数量

    plt.figure(figsize=(16, 12))
    # plt.subplot(n_rows, n_cols, plot_num)
    
    plt.subplot(2, 3, 1)
    plt.imshow(X1)
    plt.title('Original')
    plt.axis('off') 
    
    plt.subplot(2, 3, 2)
    plt.imshow(Img)
    plt.title('BC+')
    plt.axis('off') 
    
    plt.subplot(2, 3, 3)
    plt.imshow(target)
    plt.title('Target')
    plt.axis('off') 

    plt.subplot(2, 3, 4)
    plt.imshow(mask)
    plt.title('Mask')
    plt.axis('off') 
    
    plt.subplot(2, 3, 5)
    plt.imshow(dilation)
    plt.title('Prod')
    plt.axis('off') 
    
else:
    plt.subplot(2, 2, 1)
    plt.imshow(X1)
    plt.title('BC-')
    plt.axis('off') 